# Spectral Clustering on Reddit Data

In [1]:
import numpy as np
from scipy import sparse
import networkx as nx
import matplotlib.pyplot as plt
import csv

## Preprocessing

In [2]:
# import user-subreddit post behavior
accounts = {}
with open('data/reddit-user-posting-behavior.csv') as csvfile:
    reader = csv.reader(csvfile)
    idx = 0
    for row in reader:
        accounts[idx] = list(set(row[1:]))
        idx += 1

In [3]:
# get unique subreddits
subreddits = list(set([v for values in accounts.values() for v in values]))

# map subreddits to index mappings
subidx = {}
idx_to_sub = {}
for i in range(len(subreddits)):
    subidx[subreddits[i]] = i
    idx_to_sub[i] = subreddits[i]
    
# print total number of subreddits
print('Total subreddits: {}'.format(len(subreddits)))

Total subreddits: 15122


In [4]:
# prepare sparse cols and rows
row = []
col = []
for user,sublist in accounts.items():
    for sub in sublist:
        row.append(subidx[sub])
        col.append(user)

# build subreddit-user relation matrix
submat = sparse.csr_matrix((np.ones(len(row)),(row,col)))

# create final subreddit-subreddit relations
srs = submat*submat.T

In [5]:
# strip small degree
pmat = srs.toarray()
#pmat[pmat < 100] = 0

# build percentage matrix
diag = 1/srs.diagonal()
pmat = np.multiply(pmat,diag.reshape((-1,1)))

# threshold percentages
pmat[pmat < 0.05] = 0

# remove edges that are only one-sided
pmat = np.multiply(pmat, pmat.T)
pmat = pmat > 0

In [6]:
# create graph
G = nx.from_numpy_matrix(pmat, create_using=nx.Graph())

# relabel nodes
G = nx.relabel_nodes(G, idx_to_sub)

# remove isolates and self edges
G.remove_edges_from(list(G.selfloop_edges()))
G.remove_nodes_from(list(nx.isolates(G)))

In [7]:
# find largest connected component
cc = [G.subgraph(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)][0]

In [8]:
# optionally, save largest connected component to gephi file
# nx.write_gexf(cc, 'graphs/lcc.gexf')

In [9]:
def spectral_clustering(G, k):
    import numpy.linalg as la
    import scipy.cluster.vq as vq
    
    A = nx.adjacency_matrix(G)
    D = np.diag(np.ravel(np.sum(A, axis=1)))
    L = D - A
    l, U = la.eigh(L)
    f = U[:, 1]
    labels = np.ravel(np.sign(f))
    
    means, labels = vq.kmeans2(U[:, 1:k], k)

    communities = [set(np.array(list(G.nodes))[labels == l]) for l in np.unique(labels)]

    return communities

In [10]:
# perform spectral clustering
ks = np.arange(10, 110, 10)
communities = [spectral_clustering(cc, k=k) for k in ks]

C:\Users\lousc\Anaconda3\envs\py3-6\lib\site-packages\scipy\cluster\vq.py:525: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  warnings.warn("One of the clusters is empty. "


## Compute modularities

In [11]:
modularities = [nx.community.modularity(cc, community) for community in communites]
modularities

[0.5817729235860879,
 0.5583046413113081,
 0.5092365132409091,
 0.5460910766459377,
 0.4569773843090959,
 0.6827996074173764,
 0.6830314680369661,
 0.7573878048166258,
 0.623084263966161,
 0.722571779557198]

## Compute Conductances

In [16]:
def conductances(G, communities):
    '''Compute conductance for a list of communities
    '''
    return [nx.algorithms.cuts.conductance(G, community) for community in communities]

In [17]:
conductance_list = [conductances(cc, c) for c in communites]

In [39]:
[len(c) for c in communites]

[10, 19, 23, 34, 43, 56, 58, 68, 74, 74]

In [47]:
avg_conductances = []
std_conductances = []
min_conductances = []
for c in conductance_list:
    c_arr = np.array(c)
    avg_conductances.append(np.average(c_arr))
    std_conductances.append(np.std(c_arr))
    min_conductances.append(np.min(c_arr))

In [31]:
avg_conductances

[0.03069345207304954,
 0.058423343963943,
 0.05093264542583004,
 0.055343889178186056,
 0.0698631236918335,
 0.08400994111488691,
 0.05934504553416607,
 0.07303860485284937,
 0.08096740442306477,
 0.0737842990371654]

In [48]:
std_conductances

[0.019883410920741567,
 0.06034318151545104,
 0.041189166841016214,
 0.034412520929908846,
 0.05892564476467786,
 0.12994859116261573,
 0.026865829723147154,
 0.05495181958573338,
 0.06329756360546539,
 0.04529635438226902]

In [50]:
min_conductances

[0.010024269283528543,
 0.011985181956853346,
 0.014084507042253521,
 0.011235955056179775,
 0.02564102564102564,
 0.011235955056179775,
 0.01406799531066823,
 0.011235955056179775,
 0.01898333684876608,
 0.014889577295892875]